In [2]:
import os
import cv2
import glob
import pickle
import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from PIL import Image
from imutils import paths
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Convolution2D, Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization, Lambda

# from tensorflow.keras.applications import ResNet101, ResNet152, ResNet50, Xception, ResNet50V2, DenseNet201, DenseNet169, InceptionV3
from tensorflow.keras.applications import DenseNet201, ResNet50, ResNet101, Xception

In [3]:
####################
# Data Preparation #
####################

class DataGenerator():
    def __init__(self, attribute_info):
        self.attribute_info = attribute_info
        self.data_path = attribute_info['data dir']
        self.img_size = attribute_info['image size']
        
    # shuffle image paths and return shuffled image paths
    def shuffle(self):
        imagePaths = sorted(list(paths.list_images(self.data_path)))
        random.shuffle(imagePaths)

        return imagePaths
    
    # converts RGBA PNG to RGB
    def format_rgb(self, image):
        background = Image.new('RGB', image.size, (255, 255, 255))
        background.paste(image, mask = image.split()[3])
        return background

    # save labels using pickle library
    def save_data(self, data_lst):
        #exclude data array
        new_data_lst = data_lst[1:]

        #create save folder
        save_folder = os.getcwd() + '\\pickled_data'
        os.mkdir(save_folder)

        #get attribute name list
        attribute_names = list(self.attribute_info['attributes'].keys())

        #save data
        for attribute_indx in range(len(attribute_names)):
            attribute_name = attribute_names[attribute_indx]

            save_path = f'{save_folder}\\{attribute_name}.pickle'

            f = open(r'{}'.format(save_path), "wb")
            f.write(pickle.dumps(new_data_lst[attribute_indx]))
            f.close()


    # resize image and append image and its labels into respective lists
    def preprocess(self):
        img_lst = []
        age_lst = []
        mask_lst = []
        gender_lst = []
        eyewear_lst = []
        headwear_lst = []
        haircolor_lst = []
        hairlength_lst = []
        facialhair_lst = []
        
        img_size = self.img_size
        imagePaths = self.shuffle()

        for img_path_indx in tqdm(range(len(imagePaths))):
            img_path = imagePaths[img_path_indx]

            #load image, resize it, convert it to array and append to image list
            img = Image.open(img_path)
            img = img.resize(img_size)
            img_array = np.array(img)
            
            #ensure images are RGB
            if img_array.shape == (img_size[1],img_size[0], 3):
                img_lst.append(img_array)
            else:
                #format image to RGB before appending to list
                img = self.format_rgb(img) 
                img_array = np.array(img)
                img_lst.append(img_array)                

            #extract labels from path name and append to respective label lists
            (gender, eyewear, hairlength, facehair, headwear, haircolor, mask, age) = img_path.split(os.path.sep)[-2].split("_")
            age_lst.append(age)
            mask_lst.append(mask)
            gender_lst.append(gender)
            eyewear_lst.append(eyewear)
            headwear_lst.append(headwear)
            facialhair_lst.append(facehair)
            haircolor_lst.append(haircolor)
            hairlength_lst.append(hairlength)
        
        #normalize image list
        data_array = np.array(img_lst, dtype=np.float32)/255.0
        
        #convert label lists to numpy array
        age_array = np.array(age_lst)
        mask_array = np.array(mask_lst)
        gender_array = np.array(gender_lst) 
        eyewear_array = np.array(eyewear_lst) 
        headwear_array = np.array(headwear_lst) 
        haircolor_array = np.array(haircolor_lst) 
        hairlength_array = np.array(hairlength_lst) 
        facialhair_array = np.array(facialhair_lst)
        
        #binarize labels
        # NOTE: e.g  original list of labels --> [black, black, yellow, green, green, white, ...] 
        #           unique labels --> [black, green, yellow, white]
        #           one-hot encoding --> black = [1,0,0,0], green = [0,1,0,0], etc.
        #           new array of binary labels --> [ [1,0,0,0], [1,0,0,0], [0,0,1,0], [0,1,0,0], ....]
        
        age_binary_labels = LabelBinarizer().fit_transform(age_array)
        mask_binary_labels = LabelBinarizer().fit_transform(mask_array)
        gender_binary_labels = LabelBinarizer().fit_transform(gender_array)
        eyewear_binary_labels = LabelBinarizer().fit_transform(eyewear_array)
        headwear_binary_labels = LabelBinarizer().fit_transform(headwear_array)
        haircolor_binary_labels = LabelBinarizer().fit_transform(haircolor_array)
        hairlength_binary_labels = LabelBinarizer().fit_transform(hairlength_array)
        facialhair_binary_labels = LabelBinarizer().fit_transform(facialhair_array)
        
        return [data_array, gender_binary_labels, eyewear_binary_labels, hairlength_binary_labels, facialhair_binary_labels, headwear_binary_labels, haircolor_binary_labels, mask_binary_labels, age_binary_labels]
    
    #train-test split data
    def data_split(self, save_data = False):
        data_lst = data_array, gender_binary_labels, eyewear_binary_labels, hairlength_binary_labels, facialhair_binary_labels, headwear_binary_labels, haircolor_binary_labels, mask_binary_labels, age_binary_labels = self.preprocess()
        
        if save_data:
            self.save_data(data_lst)
            print('Data saved \n')

        (train_data, test_data,             # train/test data
        train_gender, test_gender,          # train/test gender labels
        train_eyewear, test_eyewear,        # train/test eyewear labels
        train_hairlength, test_hairlength,  # train/test hairlength labels
        train_facialhair, test_facialhair,  # train/test facialhair labels
        train_headwear, test_headwear,      # train/test headwear labels
        train_haircolor, test_haircolor,    # train/test haircolor labels
        train_mask, test_mask,              # train/test mask labels, train/test age labels
        train_age, test_age) = train_test_split(data_array,
                                                gender_binary_labels, eyewear_binary_labels, hairlength_binary_labels, facialhair_binary_labels, headwear_binary_labels, haircolor_binary_labels, mask_binary_labels, age_binary_labels,
                                                test_size=0.2)
        print("Data Preprocessing Completed")
        return (train_data, test_data, train_gender, test_gender, train_eyewear, test_eyewear, train_hairlength, test_hairlength, train_facialhair, test_facialhair, train_headwear, test_headwear, train_haircolor, test_haircolor, train_mask, test_mask, train_age, test_age)

In [4]:
data_path = r"D:\Daniel\research\PA-Head_Dataset"
img_size = (96,96)
generator = DataGenerator(attribute_info)
items = generator.data_split(save_data=True)
items

100%|██████████| 55235/55235 [15:20<00:00, 60.00it/s]


Data saved 

Data Preprocessing Completed


(array([[[[0.5372549 , 0.6       , 0.34509805],
          [0.5686275 , 0.6313726 , 0.3764706 ],
          [0.57254905, 0.63529414, 0.38039216],
          ...,
          [0.7058824 , 0.7254902 , 0.6509804 ],
          [0.7019608 , 0.72156864, 0.6392157 ],
          [0.7058824 , 0.7254902 , 0.63529414]],
 
         [[0.54901963, 0.6117647 , 0.3529412 ],
          [0.57254905, 0.6392157 , 0.3764706 ],
          [0.5686275 , 0.6313726 , 0.37254903],
          ...,
          [0.70980394, 0.7294118 , 0.654902  ],
          [0.7058824 , 0.7254902 , 0.6431373 ],
          [0.70980394, 0.7294118 , 0.6392157 ]],
 
         [[0.58431375, 0.6431373 , 0.3764706 ],
          [0.59607846, 0.654902  , 0.38431373],
          [0.5411765 , 0.6       , 0.33333334],
          ...,
          [0.69803923, 0.72156864, 0.65882355],
          [0.6901961 , 0.72156864, 0.63529414],
          [0.6901961 , 0.72156864, 0.627451  ]],
 
         ...,
 
         [[0.07058824, 0.07450981, 0.01960784],
          [0.14901

In [10]:
############################
# Build Multi-Output Model #
############################
class MultiOutputModel():
    #Different Head Attributes: Gender, Age, Head accessories, Hair Type(long,short,etc.), Hair Color, Eye accessories, Mouth accessories, Facial Hair
    #Approach: split color-dependent entities and non-color entities into 2 different CNN branches
        # Branch 1: ResNet50 --> for hair colour
        # Branch 2: DenseNet201 --> for head, eye, and mouth accessories [fabrics] (distinguished by shapes, sizes and textures)
        # Branch 3: ResNet101 --> for gender, age, facial hair [human-features]
    
    ##################
    # Initialisation #
    ##################
    def __init__(self, attribute_info, processed_data=None, model_path=None):

        #verify information of attribute_info dictionary
        try:
            attribute_info['attributes']
            attribute_info['image size']
            
            if processed_data:
                attribute_info['weights dir']
                attribute_info['logs dir']
                attribute_info['loss']
                attribute_info['loss weights']
                attribute_info['metrics']

            self.attribute_info = attribute_info

        except Exception as e:
            print('Attribute dictionary is not valid')

        #function to one-hot encode binary variable: due to tensorflow error for (None,2) when given (None,1) 
        def verify_encoding(labels):
            labels = list(labels)
            for label_array_indx in range(len(labels)):
                label_array = labels[label_array_indx]
                if len(label_array[0])==1: #e.g [0] or [1] --> [1,0] or [0,1]
                    new_label_array = np.hstack((1-label_array, label_array))
                    labels[label_array_indx] = new_label_array
            return tuple(labels)

        #initialize variables for training model
        if processed_data:

            #get attributes from attribute_info dictionary
            attributes = [attribute + "_output" for attribute in list(attribute_info['attributes'].keys())]

            #get train and test data
            self.train_data, self.test_data = processed_data[:2]

            #make train/test labels dictionaries
            train_labels = verify_encoding(processed_data[2::2])
            test_labels = verify_encoding(processed_data[3::2])

            self.train_labels_dict = dict(zip(attributes, train_labels))
            self.test_labels_dict = dict(zip(attributes, test_labels))


            #get model compilation information
            self.loss_dict = attribute_info['loss']
            self.loss_weights_dict = attribute_info['loss weights']
            self.metrics_dict = attribute_info['metrics']

            #Configure GPU device
            os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
            os.environ["CUDA_VISIBLE_DEVICES"] = '0'
    

        #load trained model (for inference)
        if model_path:
            print("Loading model...")

            self.model = load_model(model_path)

            print("Model sucessfully loaded")
    
    ##########################
    # Grayscale Conv2D Layer #
    ##########################
    def build_conv_grayscale_layer(self, branch_name):
        #NOTE: this layer converts 3-channel array (RGB) into 1-channel array (grayscale)
        # (m, height, width, 3)--> (m, height, width, 1)

        
        #formula for grayscale image: 0.299R + 0.587G + 0.114B
        weights = np.array([[[[ 0.299],
                            [0.587 ],
                            [0.114]]]], dtype='float32')
        
        #construct conv layer with specified weights
        conv_layer = Conv2D(1, kernel_size=(1, 1),kernel_initializer=tf.keras.initializers.Constant(value=weights), activation=None,use_bias=True,name=f'conv1_convert_grayscale_{branch_name}')
        conv_layer.trainable = False
        
        return conv_layer

    ########################################
    # Fully Connected Layer (Output Layer) #
    ########################################
    def build_fully_connected_layer(self, input_tensor, num_classes, branch_id):
        if num_classes<=2:
            activation = 'sigmoid' #binary classifier
        else:
            activation = 'softmax' #multi-class classifier
        
        x = Flatten()(input_tensor)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.5)(x)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.5)(x)
        x = Dense(num_classes, activation = activation, name= f'{branch_id}_output')(x)
        return x
    
    ##############################################
    # First CNN Branch (Colour-related features) #
    ##############################################
    def build_color_branch(self, input_tensor):
        #initialise base model
        base_model = ResNet50(input_tensor = input_tensor, include_top=False, weights='imagenet')
        
        base_model.trainable = True

        x = base_model.output
        
        return x
    
    ###################################################
    # Second CNN Branch (Non-colour-related features) #
    ###################################################
    def build_fabric_branch(self, input_tensor, uff_model = True):
        if uff_model: 
            #build grayscale conv layer
            grayscale_conv_layer = self.build_conv_grayscale_layer('fabric')
            new_input_tensor = grayscale_conv_layer(input_tensor)
            
            #initialise base model
            img_width, img_height = self.attribute_info['image size']
            base_model = DenseNet201(input_tensor = Input((img_width, img_height,1)), include_top=False, weights=None)(new_input_tensor)

            #let each layer in model to be trainable
            base_model.trainable = True

            # x = base_model.output
            x = base_model
            
            return x

        else: #Lambda layers are only usable in non-uff format
            #convert 3-channel input to a grayscale input using Tensorflow Lambda layer
            input_tensor = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(input_tensor)

            #base model is trained on RGB images (3 channels) --> make grayscale input into 3 channels
            input_tensor = Conv2D(3, (3,3), padding ='same')(input_tensor)

            #initialise base model
            base_model = DenseNet201(include_top=False, weights='imagenet')
            
            #let each layer in base model to be trainable
            base_model.trainable = True 

            #output base model after feeding 3-channel input: we cannot use model(input_tensor=input_tensor) due to additional conv2d layer that makes the model not compatible: initial n-layer NN becomes (n+1)-layer NN
            x = base_model(input_tensor)
            
            return x
    
    ##################################################
    # Third CNN Branch (Non-colour-related features) #
    ##################################################
    def build_face_branch(self, input_tensor, uff_model = True):
        if uff_model: 
            #build grayscale conv layer
            grayscale_conv_layer = self.build_conv_grayscale_layer('face')
            new_input_tensor = grayscale_conv_layer(input_tensor)
            
            #initialise base model
            img_width, img_height = self.attribute_info['image size']
            base_model = ResNet101(input_tensor = Input((img_width, img_height, 1)), include_top=False, weights=None)(new_input_tensor)

            # let each layer in model to be trainable
            base_model.trainable = True

            # x = base_model.output
            x = base_model

            return x
            
        else: #Lambda layers are usable in non-uff format
            #convert 3-channel input to a grayscale input using Tensorflow Lambda layer
            input_tensor = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(input_tensor)

            #base model is trained on RGB images (3 channels) --> make grayscale input into 3 channels
            input_tensor = Conv2D(3, (3,3), padding ='same')(input_tensor)
            
            #initialise base model
            base_model = Xception(include_top=False, weights='imagenet')

            # let each layer in model to be trainable
            base_model.trainable = True

            #output base model after feeding 3-channel input
            x = base_model(input_tensor)
            
            return x

    ################################################
    # Assemble Full Model (Combining CNN branches) #
    ################################################
    def assemble_full_model(self):
        ##################
        # Initialisation #
        ##################
        #get img width and height
        img_width, img_height = self.attribute_info['image size']
        
        #obtain attribute class lists
        gender_lst = self.attribute_info['attributes']['gender']
        age_range_lst = self.attribute_info['attributes']['age_range']
        headwear_lst = self.attribute_info['attributes']['headwear']
        hair_length_lst = self.attribute_info['attributes']['hair_length']
        hair_color_lst = self.attribute_info['attributes']['hair_color']
        eyewear_lst = self.attribute_info['attributes']['eyewear']
        maskwear_lst = self.attribute_info['attributes']['maskwear']
        facial_hair_lst = self.attribute_info['attributes']['facial_hair']
        
        #initialise input
        input_shape = (img_height, img_width, 3)
        input_tensor = Input(shape=input_shape)
        
        #build the CNN branches
        color_branch = self.build_color_branch(input_tensor)
        print('Color branch built')
        fabric_branch = self.build_fabric_branch(input_tensor)
        print('Fabric branch built')
        face_branch = self.build_face_branch(input_tensor)
        print('Face branch built')
        
        
        #################
        # Multi-Outputs #
        #################

        #color branch
        hair_color = self.build_fully_connected_layer(color_branch, len(hair_color_lst), 'hair_color')
        
        #fabric branch 
        headwear = self.build_fully_connected_layer(fabric_branch, len(headwear_lst), 'headwear')
        eyewear = self.build_fully_connected_layer(fabric_branch, len(eyewear_lst), 'eyewear')
        maskwear = self.build_fully_connected_layer(fabric_branch, len(maskwear_lst), 'maskwear')
        
        #face branch
        gender = self.build_fully_connected_layer(face_branch, len(gender_lst), 'gender')
        age_range = self.build_fully_connected_layer(face_branch, len(age_range_lst), 'age_range')
        hair_length = self.build_fully_connected_layer(face_branch, len(hair_length_lst), 'hair_length')
        facial_hair = self.build_fully_connected_layer(face_branch, len(facial_hair_lst), 'facial_hair')
        
        
        #######################
        # Assemble full model #
        #######################

        model = Model(inputs = input_tensor, 
                      outputs = [gender, eyewear, hair_length, facial_hair, headwear, hair_color, maskwear, age_range],
                      name ='Face_Attribute')

        ####################################
        # Verify uniqueness of layer names #
        ####################################
        #NOTE: duplicated names are preventing trained model to be saved in h5 format
        #       REFERENCE: https://github.com/tensorflow/tensorflow/issues/46871
        
        def verify_unique_layers(model):
            weights_names = []
            weights_duplicates = []

            cnn_layer_names = []
            cnn_layer_duplicates = []

            #check unqiueness of names of weights of embedding columns
            for i, layer in enumerate(model.weights):
                if layer.name not in weights_names:
                    weights_names.append(layer.name)
                else:
                    weights_duplicates.append((i, layer.name))
            
            #check uniqueness of names of CNN layer names
            for i, layer in enumerate(model.layers):
                if layer.name not in cnn_layer_names:
                    cnn_layer_names.append(layer.name)
                else:
                    cnn_layer_duplicates.append((i, layer.name))
            
            #change duplicated names
            if weights_duplicates:
                print("Changing duplicated names of weights")
                for i, _ in weights_duplicates:
                    model.weights[i]._handle_name = "EP_" + str(i) + "_"  + model.weights[i].name 

            if cnn_layer_duplicates:
                print("Changing duplicated names of CNN layers")
                for i, _ in cnn_layer_duplicates:
                    model.layers[i]._handle_name = "EP_" + str(i) + "_"  + model.layers[i].name
            
            return model
        
        model = verify_unique_layers(model)
            
        print('Full Model Assembled')
        return model

    ##################
    # Model Training #
    ##################
    def train(self, model, epochs):
        #define optimizer
        optimizer = Adam(learning_rate = 1e-5)

        #compile model
        model.compile(optimizer = optimizer, loss = self.loss_dict, loss_weights = self.loss_weights_dict, metrics = self.metrics_dict)

        #callbacks
        # Model Checkpointer
        checkpointer = ModelCheckpoint(filepath = os.path.join(self.attribute_info['weights dir'],"weights.{epoch:02d}-{val_accuracy:.4f}-{val_loss:.4f}.h5"), verbose=1, save_best_only=False)
        
        # Logging with Tensorboard
        tensorboard = tf.keras.callbacks.TensorBoard(
                                                    log_dir=self.attribute_info['logs dir'],
                                                    histogram_freq=0,
                                                    write_graph=True,
                                                    write_images=True,
                                                    update_freq="epoch",
                                                    profile_batch=2,
                                                    embeddings_freq=0,
                                                    embeddings_metadata=None,  
                                                    )

        # LR on Plateau 
        reduce_lr_on_plateau = ReduceLROnPlateau(
                                                monitor='val_loss', 
                                                factor=0.1, 
                                                patience=5, 
                                                verbose=1,
                                                mode='min', 
                                                min_delta=0.0001, 
                                                cooldown=0, 
                                                min_lr=0,
                                                )

        # Early Stopping
        early_stopping = EarlyStopping(monitor='val_loss', min_delta = 0.001, patience = 30, mode='min')

        #model training
        hist = model.fit(x=self.train_data,
                        y=self.train_labels_dict,
                        validation_data = (self.test_data, self.test_labels_dict),
                        epochs = epochs,
                        batch_size = 32,
                        verbose = 1,
                        callbacks = [checkpointer, tensorboard, reduce_lr_on_plateau]
                        )
        
    ####################
    # Model Prediction #
    ####################
    def predict(self, img_path):

        img_size = self.attribute_info['image size']

        #preprocess image before prediction
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_size[1],img_size[0]), cv2.INTER_LANCZOS4)
        img = img/255.
        img_array = np.array(img)
        img_array = img_array.reshape(-1, img_size[0],img_size[1],3)

        #predict image
        y_pred = self.model.predict(img_array, verbose =1 )

        lst_predict_index = []
        for arr in y_pred:
            pred_class_index = np.argmax(arr)
            lst_predict_index.append(pred_class_index)

        i = 0 
        str = ""
        for class_, attr in attribute_info['attributes'].items():
            str += class_ + ": " + attr[lst_predict_index[i]] + "\n"
            # lst_predictions.append([class_, attr[lst_predict_index[i]]])
            i+=1

        print(str)

        return 


In [5]:
attribute_info = {}

#Face Attribute classes: ensure that the attributes are in order
attribute_info['attributes'] = {}
attribute_info['attributes']['gender'] = ['Female','Male']
attribute_info['attributes']['eyewear'] = ['Glasses', 'None', 'Sunglasses', 'Unknown']
attribute_info['attributes']['hair_length'] = ['Bald', 'Long', 'Medium', 'Medium', 'Short', 'Unknown']
attribute_info['attributes']['facial_hair'] = ['Beard', 'Moustache', 'None', 'Unknown']
attribute_info['attributes']['headwear'] = ['Cap', 'Hat', 'None', 'Hat', 'Hijab', 'None', 'Hat']
attribute_info['attributes']['hair_color'] = ['Black', 'Blue', 'Brown', 'Grey', 'None', 'Red', 'Unknown', 'White']
attribute_info['attributes']['maskwear'] = ['Mask', 'None', 'Unknown']
attribute_info['attributes']['age_range'] = ['10s', '20s', '30s', '40s', 'Above 50s', 'Below 10']

#loss for each output
losses = {
    "gender_output": "binary_crossentropy",
    "eyewear_output": "categorical_crossentropy",
    "hair_length_output": "categorical_crossentropy",
    "facial_hair_output": "categorical_crossentropy",
    "headwear_output": "categorical_crossentropy",
    "hair_color_output": "categorical_crossentropy",
    "maskwear_output": "categorical_crossentropy",
    "age_range_output": "categorical_crossentropy"}

attribute_info['loss'] = losses

#loss weights for each output
loss_weights = {"gender_output": 1.0, 
                "eyewear_output": 1.0, 
                "hair_length_output": 1.0, 
                "facial_hair_output": 1.0, 
                "headwear_output": 1.0, 
                "hair_color_output": 1.0, 
                "maskwear_output": 1.0, 
                "age_range_output": 1.0}

attribute_info['loss weights'] = loss_weights

#model metrics
metrics = ["accuracy"] # use "accuracy" for all outputs
attribute_info['metrics'] = metrics

#Image size
attribute_info['image size'] = (96, 96)

#Directories
attribute_info['data dir'] = r"D:\Daniel\research\PA-Head_Dataset"
attribute_info['weights dir'] = r"E:\Daniel\Research\head_attribute\weights\model1"
attribute_info['logs dir'] = r"E:\Daniel\Research\head_attribute\logs\model1"



In [8]:
# Assemble full multi-ouput model
multioutput = MultiOutputModel(attribute_info, items)
model = multioutput.assemble_full_model()

Color branch built
Fabric branch built
Face branch built
Full Model Assembled


In [9]:
# Train model
epochs = 100
# multioutput.train(model, epochs)
p = os.getcwd() + '\\folder'
i = f'{p}'
r = r'{}'.format(i)
print(r)

d:\Daniel\research\scripts\folder


In [4]:
##############################################################################
# Trial and error to convert RGB image to grayscale image using Conv2D layer #
##############################################################################

input = r"/Users/danielng/Documents/xrvision/PSA/data/Orange Vests/Orange Vests Frames/74sim.jpg"
img1 = cv2.imread(input)
img = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
gray_img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
input_tensor = np.array(img, dtype='float32')

input_tensor = np.expand_dims(input_tensor, axis=0)
# weights = np.array([0.299, 0.587, 0.114])
weights = np.array([[[[ 0.299],
         [0.587 ],
         [0.114]]]], dtype='float32')

# x = Conv2D(1, (1,1), padding = 'same')(input_tensor)
# x = Conv2D(1, (1,1), padding='same', activation = 'relu')
# x = Conv2D(1, (1,1), padding = 'same', activation = None, bias_initializer='zeros', kernel_initializer=None, trainable = False)
x = Conv2D(1, kernel_size=(1, 1),kernel_initializer=tf.keras.initializers.zeros(), activation="relu",use_bias=True,name='conv1_convert_grayscale')
x = Conv2D(1, kernel_size=(1, 1),kernel_initializer=tf.keras.initializers.Constant(value=weights), activation=None,use_bias=True,name='conv1_convert_grayscale')

model = Sequential()
model.add(x)
# model.layers[0].trainable = False
y = model(input_tensor)
model.get_weights()
path = r'/Users/danielng/Documents/xrvision/Research/Face Attribute/data/img.jpg'

y = y[0]
y = np.array(y)
cv2.imwrite(path, y)

2021-11-29 14:36:58.141346: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


True

27 5.5699


In [11]:
####################
# Model Prediction #
####################

model_path = r"E:\Daniel\Research\head_attribute\best_weights\weights.26-5.5550.h5"
multioutput = MultiOutputModel(attribute_info, processed_data = False, model_path= model_path)


Loading model...
Model sucessfully loaded


In [16]:
img_path = r"E:\Daniel\Research\head_attribute\test_data\20191030_121201_thumb0179_0.jpg"

multioutput.predict(img_path)

1/1 [==============================] - 0s 78ms/step
gender: Male
eyewear: None
hair_length: Short
facial_hair: None
headwear: None
hair_color: Black
maskwear: None
age_range: 20s



In [77]:
path = r"E:\Daniel\Research\head_attribute\weights\model1"

best = 6
best_epoch =0
for w_path in glob.glob(path + '\*'):
    epoch = w_path.split('-')[0].split('weights.')[-1]
    val_loss = float(w_path.split('-')[-1].split('.h5')[0])
    if val_loss<best:
        best = val_loss
        best_epoch = epoch

print(best_epoch, best)

27 5.5699
